
# Tutorial: Ekstraksi Landmark Lengan untuk Boxing Pose Detection

Notebook ini menjelaskan bagaimana cara mendapatkan data landmark kedua lengan (kiri dan kanan) menggunakan MediaPipe dan menyimpannya dalam format JSON dengan koordinat XYZ beserta label pose yang sesuai.

## Overview,
- **Tujuan**: Mengekstrak landmark lengan kiri dan kanan untuk deteksi pose boxing
- **Output**: File JSON berisi koordinat XYZ landmark dengan label pose
- **Library**: MediaPipe, OpenCV, JSON

## 1. Import Library yang Diperlukan

In [ ]:
import cv2
import mediapipe as mp
import json
import os
import datetime
import numpy as np
from IPython.display import display, Image as IPImage
import matplotlib.pyplot as plt

print("Library berhasil diimport!")

## 2. Setup MediaPipe Pose Detection

MediaPipe menyediakan 33 landmark pose. Untuk boxing pose, kita fokus pada landmark lengan:

In [ ]:
# Inisialisasi MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Setup pose detector dengan parameter optimal
pose = mp_pose.Pose(
    static_image_mode=False,         # Mode real-time
    model_complexity=1,              # Kompleksitas model (0-2)
    min_detection_confidence=0.7,    # Minimum confidence untuk deteksi
    min_tracking_confidence=0.5      # Minimum confidence untuk tracking
)

print("MediaPipe Pose berhasil diinisialisasi!")

## 3. Definisi Landmark Lengan

MediaPipe Pose memberikan 33 landmark. Untuk boxing pose, kita fokus pada landmark upper body terutama lengan:

In [ ]:
# Mapping landmark MediaPipe untuk upper body
LANDMARK_MAPPING = {
    # Bahu
    11: 'LEFT_SHOULDER',
    12: 'RIGHT_SHOULDER',
    
    # Siku
    13: 'LEFT_ELBOW',
    14: 'RIGHT_ELBOW',
    
    # Pergelangan tangan
    15: 'LEFT_WRIST',
    16: 'RIGHT_WRIST'
}

# Index landmark yang akan diekstrak (upper body focus)
UPPER_BODY_INDICES = [11, 12, 13, 14, 15, 16]

print("Landmark mapping:")
for idx, name in LANDMARK_MAPPING.items():
    print(f"Index {idx}: {name}")


## 4. Fungsi Ekstraksi Landmark

Fungsi ini mengekstrak koordinat XYZ dari landmark lengan kiri dan kanan:

In [ ]:
def extract_arm_landmarks(landmarks):
    """
    Ekstraksi landmark lengan kiri dan kanan dari hasil MediaPipe
    
    Args:
        landmarks: MediaPipe landmarks object
    
    Returns:
        dict: Dictionary berisi landmark lengan kiri dan kanan dengan koordinat XYZ
    """
    
    arm_data = {
        'left_arm': {},
        'right_arm': {},
        'timestamp': datetime.datetime.now().isoformat()
    }
    
    # Ekstrak landmark untuk setiap bagian lengan
    for idx in UPPER_BODY_INDICES:
        landmark = landmarks[idx]
        landmark_name = LANDMARK_MAPPING[idx]
        
        # Koordinat XYZ (normalized 0-1)
        coord_data = {
            'x': float(landmark.x),        # Koordinat horizontal (0-1)
            'y': float(landmark.y),        # Koordinat vertikal (0-1)
            'z': float(landmark.z),        # Kedalaman relatif
            'visibility': float(landmark.visibility)  # Tingkat visibilitas (0-1)
        }
        
        # Kategorisasi berdasarkan lengan kiri/kanan
        if 'LEFT' in landmark_name:
            part_name = landmark_name.replace('LEFT_', '').lower()
            arm_data['left_arm'][part_name] = coord_data
        elif 'RIGHT' in landmark_name:
            part_name = landmark_name.replace('RIGHT_', '').lower()
            arm_data['right_arm'][part_name] = coord_data
    
    return arm_data

print("Fungsi ekstraksi landmark berhasil didefinisikan!")


## 5. Fungsi Capture dan Penyimpanan Data

Fungsi untuk menangkap pose dan menyimpan data landmark dalam format JSON:

In [ ]:
def setup_directories():
    """Setup direktori untuk menyimpan data pose"""
    gesture_dir = 'boxing_poses'
    if not os.path.exists(gesture_dir):
        os.makedirs(gesture_dir)
        print(f"Direktori {gesture_dir} berhasil dibuat!")
    
    return gesture_dir

def save_pose_data(arm_landmarks, pose_label, image_path, gesture_dir):
    """
    Menyimpan data landmark pose ke file JSON

    Args:
        arm_landmarks: Data landmark lengan
        pose_label: Label pose (JAB, HOOK, UPPERCUT)
        image_path: Path ke file gambar
        gesture_dir: Direktori penyimpanan
    """
    
    json_file = os.path.join(gesture_dir, 'boxing_poses.json')
    
    # Load existing data atau buat baru
    if os.path.exists(json_file):
        with open(json_file, 'r') as f:
            gesture_db = json.load(f)
    else:
        gesture_db = []
    
    # Buat entry baru
    pose_entry = {
        'id': len(gesture_db) + 1,
        'pose_label': pose_label,
        'landmarks': arm_landmarks,
        'image_path': image_path,
        'created_at': datetime.datetime.now().isoformat()
    }
    
    # Tambahkan ke database
    gesture_db.append(pose_entry)
    
    # Simpan ke file JSON
    with open(json_file, 'w') as f:
        json.dump(gesture_db, f, indent=2)
    
    print(f"Data pose '{pose_label}' berhasil disimpan! Total data: {len(gesture_db)}")
    
    return json_file

print("Fungsi penyimpanan data berhasil didefinisikan!")

## 6. Demo: Ekstraksi Landmark dari Webcam

Contoh implementasi untuk mengekstrak landmark dari webcam:

In [ ]:
def capture_pose_from_webcam(pose_label='JAB', num_captures=1):
    """
    Capture pose dari webcam dan ekstrak landmark

    Args:
        pose_label: Label pose yang akan dicapture
        num_captures: Jumlah capture yang diinginkan
    """

    # Setup direktori
    gesture_dir = setup_directories()

    # Inisialisasi webcam
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Tidak dapat mengakses webcam!")
        return

    print(f"\nMulai capture pose '{pose_label}'...")
    print("Tekan 's' untuk capture, 'q' untuk quit")

    captured_count = 0

    while captured_count < num_captures:
        ret, frame = cap.read()
        if not ret:
            print("Error: Tidak dapat membaca frame dari webcam!")
            break

        # Convert BGR ke RGB untuk MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Deteksi pose
        results = pose.process(rgb_frame)

        # Gambar landmark jika terdeteksi
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS
            )

            # Status: Pose terdeteksi
            cv2.putText(frame, f"Pose: {pose_label} - DETECTED",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            # Status: Pose tidak terdeteksi
            cv2.putText(frame, f"Pose: {pose_label} - NOT DETECTED",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Instruksi
        cv2.putText(frame, f"Captured: {captured_count}/{num_captures}",
                    (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, "Press 's' to capture, 'q' to quit",
                    (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Tampilkan frame
        cv2.imshow('Boxing Pose Capture', frame)

        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):  # Capture pose
            if results.pose_landmarks:
                # Ekstrak landmark lengan
                arm_landmarks = extract_arm_landmarks(results.pose_landmarks.landmark)

                # Simpan gambar
                timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
                img_filename = f'{pose_label}_{timestamp}_{captured_count + 1}.jpg'
                img_path = os.path.join(gesture_dir, img_filename)
                cv2.imwrite(img_path, frame)

                # Simpan data landmark
                json_file = save_pose_data(arm_landmarks, pose_label, img_path, gesture_dir)

                captured_count += 1
                print(f"Capture {captured_count}/{num_captures} berhasil!")
            else:
                print("Tidak ada pose yang terdeteksi! Coba lagi.")

        elif key == ord('q'):  # Quit
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

    print(f"\nCapture selesai! Total data tersimpan: {captured_count}/{num_captures}")

print("Fungsi capture webcam berhasil didefinisikan!")


## 7. Analisis Data Landmark

Fungsi untuk menganalisis dan memvisualisasikan data landmark yang telah tersimpan:

In [ ]:
def analyze_saved_landmarks(json_file='boxing_poses/boxing_poses.json'):
    """
    Menganalisis data landmark yang telah tersimpan

    Args:
        json_file: Path ke file JSON yang berisi data landmark
    """
    
    if not os.path.exists(json_file):
        print(f"File {json_file} tidak ditemukan!")
        return

    # Load data
    with open(json_file, 'r') as f:
        gesture_db = json.load(f)

    print(f"\n=== ANALISIS DATA LANDMARK ===")
    print(f"Total data tersimpan: {len(gesture_db)}")

    # Hitung distribusi pose
    pose_counts = {}
    for entry in gesture_db:
        pose_label = entry['pose_label']
        pose_counts[pose_label] = pose_counts.get(pose_label, 0) + 1

    print("\nDistribusi pose:")
    for pose, count in pose_counts.items():
        print(f"  {pose}: {count} data")

    # Tampilkan contoh data
    if gesture_db:
        print("\n=== CONTOH DATA LANDMARK ===")
        sample = gesture_db[0]
        print(f"Pose: {sample['pose_label']}")
        print(f"Timestamp: {sample['created_at']}")

        print("\nLandmark Lengan Kiri:")
        for part, coords in sample['landmarks']['left_arm'].items():
            print(f"  {part.upper()}: x={coords['x']:.3f}, y={coords['y']:.3f}, z={coords['z']:.3f}")

        print("\nLandmark Lengan Kanan:")
        for part, coords in sample['landmarks']['right_arm'].items():
            print(f"  {part.upper()}: x={coords['x']:.3f}, y={coords['y']:.3f}, z={coords['z']:.3f}")

    return gesture_db


def visualize_pose_distribution(gesture_db):
    """
    Visualisasi distribusi pose dalam bentuk bar chart
    """
    
    # Hitung distribusi
    pose_counts = {}
    for entry in gesture_db:
        pose_label = entry['pose_label']
        pose_counts[pose_label] = pose_counts.get(pose_label, 0) + 1

    # Buat plot
    plt.figure(figsize=(10, 6))
    poses = list(pose_counts.keys())
    counts = list(pose_counts.values())

    bars = plt.bar(poses, counts, color=['#43a047', '#e53935', '#1e88e5'])
    plt.title('Distribusi Data Boxing Pose', fontsize=16, fontweight='bold')
    plt.xlabel('Jenis Pose', fontsize=12)
    plt.ylabel('Jumlah Data', fontsize=12)

    # Tambahkan label di atas bar
    for bar, count in zip(bars, counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                 str(count), ha='center', va='bottom', fontweight='bold')

    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()


print("Fungsi analisis data berhasil didefinisikan!")


 "## 8. Contoh Penggunaan\n",
"\n",
"Berikut adalah contoh cara menggunakan fungsi-fungsi yang telah dibuat:"

In [ ]:
# Contoh 1: Capture 3 pose JAB dari webcam
capture_pose_from_webcam(pose_label='JAB', num_captures=3)

: 

In [ ]:
"# Contoh 2: Capture pose HOOK\n",
capture_pose_from_webcam(pose_label='HOOK', num_captures=2)

In [ ]:
"# Contoh 3: Capture pose UPPERCUT\n",
capture_pose_from_webcam(pose_label='UPPERCUT', num_captures=2)